In [1]:
from PIL import Image
import os, glob
import numpy as np
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES =True

clsdir = "/Users/nisi2/Desktop/cluster/"
# classes = ["hakidamegiku", "hamasuge (wild)","hosoaogeitou","inubie","inutade","kosendagusa","mehishiba","ohisiba","oinutade","shiroza"]
classes = ["mehishiba", "oinutade"]
num_classes = len(classes)
image_size = 28
num_testdata = 100

X_train = []
X_test = []
y_train = []
y_test = []
for index, classlabel in enumerate(classes):
    photos_dir = clsdir + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        # それぞれ400枚まで取得
        if i >= 400:
            break
        image = Image.open(file)
        image = image.convert("RGB")
  #      image = image.convert("L")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:
            for angle in range(-20, 20, 5):
                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)

                img_trans = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trans)
                X_train.append(data)
                y_train.append(index)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./weed.npy", xy)

In [2]:
print(X_test.shape)
print(X_train.shape)
print(y_test.shape)
print(y_train.shape)
for angle in range(-20, 20, 5):
    print(angle,end=",")
print()
print(Image.FLIP_LEFT_RIGHT)

import matplotlib.pyplot as plt

plt.imshow(X_train[0])

(200, 28, 28, 3)
(9600, 28, 28, 3)
(200,)
(9600,)
-20,-15,-10,-5,0,5,10,15,
0


In [3]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.summary()



Using TensorFlow backend.






_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
______

In [5]:
from keras.utils import np_utils
import numpy as np


# classes = ["hakidamegiku", "hamasuge (wild)","hosoaogeitou","inubie","inutade","kosendagusa","mehishiba","ohisiba","oinutade","shiroza"]
classes = ["mehishiba", "oinutade"]
num_classes = len(classes)
image_size = 28

train_images, test_images, train_labels, test_labels = np.load("./weed.npy", allow_pickle=True)

train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(test_labels, num_classes)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
9600/9600 [==============================] - 6s 662us/step - loss: 0.1219 - acc: 0.9570
Epoch 2/5
9600/9600 [==============================] - 7s 694us/step - loss: 0.0722 - acc: 0.9738
Epoch 3/5
9600/9600 [==============================] - 6s 627us/step - loss: 0.0595 - acc: 0.9779
Epoch 4/5
9600/9600 [==============================] - 6s 646us/step - loss: 0.0404 - acc: 0.9851
Epoch 5/5
9600/9600 [==============================] - 6s 672us/step - loss: 0.0444 - acc: 0.9868


In [7]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
9600/9600 [==============================] - 7s 733us/step - loss: 0.0291 - acc: 0.9920
Epoch 2/5
9600/9600 [==============================] - 7s 765us/step - loss: 0.0283 - acc: 0.9913
Epoch 3/5
9600/9600 [==============================] - 7s 722us/step - loss: 0.0264 - acc: 0.9925
Epoch 4/5
9600/9600 [==============================] - 7s 752us/step - loss: 0.0320 - acc: 0.9919 1s - loss: 0.
Epoch 5/5
9600/9600 [==============================] - 7s 728us/step - loss: 0.0220 - acc: 0.9938


In [6]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

200/200 [==============================] - 0s 743us/step


0.86

In [ ]:
history = model.fit_generator(train_gen_reverse,
                              steps_per_epoch=500,
                              epochs=20,
                              validation_data=val_gen_reverse,
                              validation_steps=val_steps)
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()